In [1]:
import sys
import pathlib
sys.path.append('../../../build')

import numpy as np
import libry as ry

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/02-kinematics/solutions'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}
** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/02-kinematics/solutions'


** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [3]:
# Here we do not need a simulation world

# adding a configuration world

C = ry.Config()

C.addFile("../../../scenarios/pandasTable.g")
print(C.getJointState())
q0 = C.getJointState()
    
D = C.view()

[ 0.  -0.5  0.  -2.   0.   2.  -0.5  0.  -0.5  0.  -2.   0.   2.  -0.5]


In [8]:
obj = C.addFrame("object")



obj.setPosition([1., 0., 1.5])

obj.setQuaternion([1,0,1,0])

obj.setShape(ry.ST.capsule, [.2,.02])

obj.setColor([1,0,1,0.5])

objFrame = C.addFrame("objectFrame",parent="object")
objFrame.setShape(ry.ST.marker,[.1])




In [9]:
obj.setPosition([.2,.2,1.5]) 


IK = C.komo_IK(False)


# uncomment for quaternion diff
# IK.addObjective([1.], 
#                 ry.FS.quaternionDiff, 
#                 ["object", "R_gripperCenter"], 
#                 ry.OT.eq, 
#                 [1e2]
#                );

IK.addObjective([1.], 
                 ry.FS.scalarProductYZ, 
                 [ "R_gripperCenter", "object" ], 
                 ry.OT.eq, 
                 [1e2],
                [1]
                );
# NOTE: we can also use:
# IK.addObjective([1.], 
#                  ry.FS.scalarProductXZ, 
#                  [ "R_gripperCenter", "object" ], 
#                  ry.OT.eq, 
#                  [1e2],
#                 [0.]
#                 );

# IK.addObjective([1.], 
#                  ry.FS.scalarProductZZ, 
#                  [ "R_gripperCenter", "object" ], 
#                  ry.OT.eq, 
#                  [1e2]
#                 );

pre_grasp_distance = .1
IK.addObjective([1.], 

                ry.FS.positionRel, 

                ["object", "R_gripperCenter"], 

                ry.OT.eq, 

                [1e2],

                [0.,.0,-pre_grasp_distance]);



In [10]:
IK.optimize()
IK.getReport()

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:226


{'F_qItself/1-#28': {'order': 1.0,
  'type': 'sos',
  'sos': 5.3828096355559135e-06},
 'F_qQuaternionNorms/0-#113': {'order': 0.0, 'type': 'eq'},
 'F_ScalarProduct/0-R_gripperCenter-object': {'order': 0.0,
  'type': 'eq',
  'eq': 0.004306346665106098},
 'F_PositionRel/0-object-R_gripperCenter': {'order': 0.0,
  'type': 'eq',
  'eq': 0.006630912221344644},
 'sos': 5.3828096355559135e-06,
 'ineq': 0.0,
 'eq': 0.010937258886450741,
 'f': 0.0}

** optimization time:0.000221 (kin:1.2e-05 coll:0 feat:7.8e-05 newton: 2e-05) setJointStateCount:3
   sos:5.38281e-06 ineq:0 eq:0.0109373


In [11]:
C.setFrameState( IK.getFrameState(0) )
print(C.getJointState())

[ 0.00544713 -0.50745285 -0.01366458 -2.00051079 -0.0055751   2.00130796
 -0.49418811  0.03605301 -0.30392017  0.32461019 -1.51509777 -0.3390449
  2.27069433 -0.31911074]
